In [10]:
import sys
sys.path.insert(1, '/trinity/home/xwan/MPNST_DL/working')
from dataset import MPNSTDataMoule, MPNSTDataset
from networks import spp
from networks.resnet3d_spp import generate_model
from torchsummary import summary
import torch

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
model = generate_model(10)

In [5]:
test_data = torch.rand(1, 512, 512, 60)

In [8]:
summary(model)

Layer (type:depth-idx)                   Param #
├─Conv3d: 1-1                            21,952
├─BatchNorm3d: 1-2                       128
├─ReLU: 1-3                              --
├─Sequential: 1-4                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv3d: 3-1                  110,592
|    |    └─BatchNorm3d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv3d: 3-4                  110,592
|    |    └─BatchNorm3d: 3-5             128
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-2                   --
|    |    └─Conv3d: 3-6                  221,184
|    |    └─BatchNorm3d: 3-7             256
|    |    └─ReLU: 3-8                    --
|    |    └─Conv3d: 3-9                  442,368
|    |    └─BatchNorm3d: 3-10            256
|    |    └─Sequential: 3-11             8,448
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv3d: 3-12               

Layer (type:depth-idx)                   Param #
├─Conv3d: 1-1                            21,952
├─BatchNorm3d: 1-2                       128
├─ReLU: 1-3                              --
├─Sequential: 1-4                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv3d: 3-1                  110,592
|    |    └─BatchNorm3d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv3d: 3-4                  110,592
|    |    └─BatchNorm3d: 3-5             128
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-2                   --
|    |    └─Conv3d: 3-6                  221,184
|    |    └─BatchNorm3d: 3-7             256
|    |    └─ReLU: 3-8                    --
|    |    └─Conv3d: 3-9                  442,368
|    |    └─BatchNorm3d: 3-10            256
|    |    └─Sequential: 3-11             8,448
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv3d: 3-12               

In [12]:
def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                    out_planes,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                    out_planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False)

In [29]:
from networks.resnet3d_spp import BasicBlock, Bottleneck
from networks.spp import SPP3DLayer

def _make_layer(block, planes, blocks, in_planes, stride=1):
        downsample = None
        if stride != 1 or in_planes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1x1(in_planes, planes * block.expansion, stride),
                nn.BatchNorm3d(planes * block.expansion)
            )
        layers = []
        layers.append(
            block(in_planes=in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        in_planes = planes * block.expansion
        for i in range (1, blocks):
            layers.append(block(in_planes, planes))
        
        return nn.Sequential(*layers)

In [14]:
block_inplanes = [int(x * 1.0) for x in get_inplanes()]

In [37]:
n_input_channels = 1
conv1_t_size = 7
conv1_t_stride = 2
scales = [5, 3, 1]
layers = [1, 1, 1, 1]
n_classes = 2

In [49]:
in_planes = block_inplanes[0]
conv1 = nn.Conv3d(n_input_channels, 
                        in_planes,
                        kernel_size=(conv1_t_size, 7, 7),
                        stride=(conv1_t_stride, 2, 2),
                        padding=(conv1_t_size // 2, 3, 3),
                        bias=False)
bn1 = nn.BatchNorm3d(in_planes)
relu = nn.ReLU(inplace=True)
block = BasicBlock
layer1 = _make_layer(BasicBlock, block_inplanes[0], layers[0], in_planes=in_planes)
layer2 = _make_layer(BasicBlock, block_inplanes[1], layers[1], in_planes=block_inplanes[1], stride=2)
layer3 = _make_layer(BasicBlock, block_inplanes[2], layers[2], in_planes=block_inplanes[2], stride=2)
layer4 = _make_layer(BasicBlock, block_inplanes[3], layers[3], in_planes=block_inplanes[3], stride=2)
spp = SPP3DLayer(scales)
linear = nn.Linear(64*sum([x**3 for x in scales]), n_classes)

In [44]:
print(test_data.shape)

x = conv1(test_data)
print(x.shape)

x = bn1(x)
print(x.shape)

x = relu(x)

torch.Size([1, 1, 512, 512, 60])
torch.Size([1, 64, 256, 256, 30])
torch.Size([1, 64, 256, 256, 30])


In [45]:
x = layer1(x)
print(x.shape)


torch.Size([1, 64, 256, 256, 30])


In [52]:
layer2

Sequential(
  (0): BasicBlock(
    (conv1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv3d(128, 128, kernel_size=(1, 1, 1), stride=(2, 2, 2), bias=False)
      (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [48]:
x = layer2(x)
print(x.shape)


RuntimeError: Given groups=1, weight of size [128, 128, 3, 3, 3], expected input[1, 64, 256, 256, 30] to have 128 channels, but got 64 channels instead